In [25]:
import pandas as pd
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


In [26]:
columns = ['Player','Date','Team','OPP','FG','FGA','FG%','3P','3PA','3P%','FT','FTA',
          'FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS',
          '+/-','TS%','eFG%','3PAr','FTr','ORB%','DRB%','TRB%','AST%',
          'STL%','BLK%','TOV%','USG','ORtg','DRtg','BPM','MP','Starter']

def _format(data, columns):
    df = pd.DataFrame(columns = columns)

    months = list(data.keys())
    player_data = []
    for month in months:

        days = list(data[month].keys())

        for day in (days):

            games = list(data[month][day].keys())
            for game in games:


                teams = list(data[month][day][game].keys())
                teams.remove('Inactive')

                for team in teams:
                    players = list(data[month][day][game][team].keys())
                    players.remove('Team')

                    for _team in teams:
                        if _team != team:
                            opp = _team
                    for player in players:
                        data_player = [player, day, team, opp] + data[month][day][game][team][player]
                        if len(data_player) > 5:
                            player_data.append(data_player)

    df = pd.DataFrame(player_data)
    df.columns = columns
    df['Date'] = pd.to_datetime(df['Date'])
    return df

years = ['2019','2020','2021']
df_list = []
for year in years:
    with open('{}.json'.format(year), 'r') as fp:
        data = json.load(fp)
        df_year = _format(data, columns)
        df_list.append(df_year)

df = pd.concat(df_list)
df

,Player,Date,Team,OPP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG,ORtg,DRtg,BPM,MP,Starter
0,Al Horford,2018-10-16,BOS,PHI,4.0,7.0,0.571,0.0,1.0,0.000,1.0,1.0,1.0,0.0,4.0,4.0,2.0,0.0,4.0,3.0,1.0,9.0,9.0,0.605,0.571,0.143,0.143,0.0,13.1,6.3,9.0,0.0,10.5,28.7,14.3,90.0,80.0,2.5,29:57,1
1,Kyrie Irving,2018-10-16,BOS,PHI,2.0,14.0,0.143,1.0,8.0,0.125,2.0,2.0,1.0,0.0,4.0,4.0,7.0,0.0,0.0,3.0,1.0,7.0,9.0,0.235,0.179,0.571,0.143,0.0,13.5,6.5,29.9,0.0,0.0,16.8,25.3,63.0,89.0,-9.0,29:00,1
2,Jayson Tatum,2018-10-16,BOS,PHI,9.0,17.0,0.529,1.0,5.0,0.200,4.0,5.0,0.8,2.0,7.0,9.0,3.0,1.0,0.0,1.0,2.0,23.0,6.0,0.599,0.559,0.294,0.294,6.3,23.7,14.6,18.4,1.6,0.0,5.0,28.6,125.0,83.0,11.7,28:56,1
3,Jaylen Brown,2018-10-16,BOS,PHI,5.0,13.0,0.385,1.0,3.0,0.333,1.0,2.0,0.5,3.0,2.0,5.0,2.0,0.0,0.0,2.0,4.0,12.0,11.0,0.432,0.423,0.231,0.154,9.6,6.9,8.3,10.1,0.0,0.0,12.6,22.9,91.0,89.0,-3.8,28:23,1
4,Gordon Hayward,2018-10-16,BOS,PHI,4.0,12.0,0.333,1.0,3.0,0.333,1.0,2.0,0.5,2.0,3.0,5.0,0.0,4.0,0.0,0.0,1.0,10.0,10.0,0.388,0.375,0.250,0.167,7.4,11.9,9.6,0.0,7.5,0.0,0.0,21.4,90.0,74.0,5.7,24:37,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2213,Hassan Whiteside,2021-01-04,SAC,GSW,2.0,5.0,0.400,0.0,0.0,NaN,4.0,5.0,0.8,1.0,1.0,2.0,0.0,0.0,3.0,1.0,0.0,8.0,-9.0,0.556,0.400,0.000,1.000,7.3,9.3,8.2,0.0,0.0,22.7,12.2,27.6,106.0,107.0,0.8,11:46,0
2214,Justin James,2021-01-04,SAC,GSW,2.0,3.0,0.667,0.0,1.0,0.000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.581,0.667,0.333,0.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.5,105.0,131.0,-6.0,7:47,0
2215,Chimezie Metu,2021-01-04,SAC,GSW,3.0,4.0,0.750,1.0,1.0,1.000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,0.875,0.875,0.250,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.4,157.0,131.0,22.1,4:45,0
2216,Robert Woodard,2021-01-04,SAC,GSW,2.0,6.0,0.333,0.0,1.0,0.000,0.0,0.0,NaN,3.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.333,0.333,0.167,0.000,54.1,68.9,60.6,0.0,0.0,0.0,0.0,50.0,91.0,127.0,-5.1,4:45,0


In [27]:
teams = df['Team'].unique()

df_list = []
for team in tqdm(teams):
    df_team = df[df['Team'] == team]
    df_opp  = df[df['OPP']  == team]

    team_list = []
    opp_list = []
    dates = df_team['Date'].unique()
    for date in dates:

        df_team_mid = df_team[df_team['Date'] == date]
        df_opp_mid = df_opp[df_opp['Date'] == date]


        team_PTS = df_team_mid['PTS'].sum()
        team_TRB = df_team_mid['TRB'].sum()
        team_AST = df_team_mid['AST'].sum()
        team_BLK = df_team_mid['BLK'].sum()
        team_TOV = df_team_mid['TOV'].sum()
        team_FT = df_team_mid['FT'].sum()
        team_ortg = df_team_mid['ORtg'].mean()
        team_drtg = df_team_mid['DRtg'].mean()

        opp_PTS = df_opp_mid['PTS'].sum()
        opp_TRB = df_opp_mid['TRB'].sum()
        opp_AST = df_opp_mid['AST'].sum()
        opp_BLK = df_opp_mid['BLK'].sum()
        opp_TOV = df_opp_mid['TOV'].sum()
        opp_FT = df_opp_mid['FT'].sum()
        opp_ortg = df_opp_mid['ORtg'].mean()
        opp_drtg = df_opp_mid['DRtg'].mean()

        team_stats = [team, date, team_PTS,team_TRB,team_AST,team_BLK,team_TOV,team_FT,team_ortg,team_drtg,
                     opp_PTS,opp_TRB,opp_AST,opp_BLK,opp_TOV,opp_FT,opp_ortg,opp_drtg]
        team_list.append(team_stats)

    df_team_final = pd.DataFrame(team_list)
    df_team_final.columns = ['TEAM','DATE','TEAM_PTS','TEAM_TRB','TEAM_AST','TEAM_BLK','TEAM_TOV','TEAM_FT','TEAM_ORTG','TEAM_DRTG',
                            'OPP_PTS','OPP_TRB','OPP_AST','OPP_BLK','OPP_TOV','OPP_FT','OPP_ORTG','OPP_DRTG']
    df_list.append(df_team_final)
    
df_team = pd.concat(df_list)
df_team

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [00:18<00:00,  1.61it/s]


,TEAM,DATE,TEAM_PTS,TEAM_TRB,TEAM_AST,TEAM_BLK,TEAM_TOV,TEAM_FT,TEAM_ORTG,TEAM_DRTG,OPP_PTS,OPP_TRB,OPP_AST,OPP_BLK,OPP_TOV,OPP_FT,OPP_ORTG,OPP_DRTG
0,BOS,2018-10-16,105.0,55.0,21.0,5.0,14.0,10.0,85.846154,83.000000,87.0,47.0,18.0,5.0,16.0,14.0,69.727273,104.000000
1,BOS,2018-10-19,101.0,49.0,29.0,6.0,14.0,7.0,101.444444,115.888889,113.0,49.0,24.0,3.0,12.0,14.0,106.000000,103.200000
2,BOS,2018-10-20,103.0,47.0,21.0,2.0,15.0,28.0,105.363636,103.454545,101.0,46.0,23.0,1.0,16.0,17.0,101.333333,105.583333
3,BOS,2018-10-22,90.0,51.0,21.0,3.0,12.0,7.0,78.000000,98.416667,93.0,49.0,28.0,2.0,9.0,7.0,84.555556,93.444444
4,BOS,2018-10-25,101.0,45.0,19.0,1.0,11.0,24.0,93.800000,96.700000,95.0,57.0,19.0,6.0,16.0,14.0,104.600000,101.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,LAL,2020-12-27,127.0,46.0,32.0,14.0,17.0,14.0,130.357143,85.714286,91.0,48.0,24.0,6.0,17.0,9.0,97.384615,117.384615
177,LAL,2020-12-28,107.0,41.0,20.0,5.0,14.0,15.0,97.400000,114.800000,115.0,45.0,27.0,6.0,9.0,13.0,127.666667,107.555556
178,LAL,2020-12-30,121.0,47.0,25.0,3.0,17.0,15.0,122.416667,109.583333,107.0,39.0,25.0,4.0,10.0,17.0,88.250000,122.916667
179,LAL,2021-01-01,109.0,58.0,27.0,7.0,13.0,13.0,102.400000,102.200000,103.0,45.0,23.0,8.0,13.0,18.0,91.333333,109.666667


In [28]:
df.to_csv("Data.csv")
df_team.to_csv("Team.csv")